In [ ]:
!pip install earthaccess


In [2]:
import earthaccess

auth = earthaccess.login(strategy="interactive")


In [3]:
# ============================================================================
# DOWNLOAD JULY 2024 GLM DATA TO GOOGLE DRIVE - Lightning Folder
# ============================================================================

# Primero instala earthaccess si no lo tienes
# !pip install earthaccess

import earthaccess
from pathlib import Path
from datetime import datetime
import time

print("=" * 70)
print("GLM DOWNLOAD TO GOOGLE DRIVE/Lightning - JULY 2024")
print("=" * 70 + "\n")

# ----------------------------------------------------------------------------
# AUTHENTICATION
# ----------------------------------------------------------------------------

print("🔐 Authenticating with NASA Earthdata...")
auth = earthaccess.login(strategy="interactive")
print("✅ Authentication successful!\n")

# ----------------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------------

COLLECTION_ID = "C2278812167-GHRC_DAAC"

# USA Continental bounding box
lat_min, lat_max = 20, 55
lon_min, lon_max = -130, -60
BBOX = (lon_min, lat_min, lon_max, lat_max)

TARGET_YEAR = 2024
TARGET_MONTH = 7
NUM_DAYS = 31

# Tu carpeta de Google Drive "Lightning"
BASE_DIR = Path("G:/My Drive/Lightning/glm_raw")

BASE_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("CONFIGURATION")
print("=" * 70)
print(f"📅 Target: July {TARGET_YEAR} (31 days)")
print(f"🗺️  Region: USA Continental {BBOX}")
print(f"📁 Output: G:/My Drive/Lightning/glm_raw/2024/07")
print(f"💾 Expected size: ~40-50 GB")
print(f"⏱️  Estimated time: 4-6 hours")
print(f"📊 Available space in Drive: ~94 GB")
print("=" * 70 + "\n")

# ----------------------------------------------------------------------------
# SEARCH FUNCTION
# ----------------------------------------------------------------------------

def search_glm_day(year, month, day, use_bbox=True):
    """Search for GLM L3 granules for a specific day."""
    start = f"{year}-{month:02d}-{day:02d}T00:00:00Z"
    end = f"{year}-{month:02d}-{day:02d}T23:59:59Z"

    search_params = {
        "concept_id": COLLECTION_ID,
        "temporal": (start, end),
        "count": 2000
    }

    if use_bbox:
        search_params["bounding_box"] = BBOX

    results = earthaccess.search_data(**search_params)
    return results

# ----------------------------------------------------------------------------
# DOWNLOAD TO GOOGLE DRIVE
# ----------------------------------------------------------------------------

print("🚀 Starting download to Google Drive/Lightning folder\n")
print("=" * 70 + "\n")

out_dir = BASE_DIR / f"{TARGET_YEAR}" / f"{TARGET_MONTH:02d}"
out_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Files will be saved to:")
print(f"   {out_dir}\n")

total_files_downloaded = 0
total_files_existing = 0
failed_days = []
start_time = time.time()

for day in range(1, NUM_DAYS + 1):
    day_start = time.time()

    print(f"📅 Day {day:02d}/{NUM_DAYS}: July {day}, {TARGET_YEAR}")
    print("-" * 70)

    try:
        print(f"🔍 Searching...")
        results = search_glm_day(TARGET_YEAR, TARGET_MONTH, day, use_bbox=True)

        if len(results) == 0:
            print(f"⚠️  No data found\n")
            failed_days.append(f"July {day}")
            continue

        print(f"✅ Found: {len(results)} files")

        # Check existing files
        day_of_year = datetime(TARGET_YEAR, TARGET_MONTH, day).timetuple().tm_yday
        existing_pattern = f"*_s{TARGET_YEAR}{day_of_year:03d}*.nc"
        existing_files = list(out_dir.glob(existing_pattern))

        if len(existing_files) >= len(results) * 0.95:
            print(f"✓ Already downloaded ({len(existing_files)} files)")
            total_files_existing += len(existing_files)
            print(f"⏭️  Skipping to next day\n")
            continue

        # Download to Google Drive
        print(f"⬇️  Downloading to Google Drive/Lightning...")
        files = earthaccess.download(results, out_dir.as_posix())

        downloaded_count = len(files)
        total_files_downloaded += downloaded_count

        day_elapsed = time.time() - day_start
        print(f"✅ Downloaded {downloaded_count} files in {day_elapsed/60:.1f} minutes")

        # Progress summary
        total_so_far = total_files_downloaded + total_files_existing
        expected_so_far = day * 1440
        progress_pct = (day / NUM_DAYS) * 100

        print(f"📊 Progress: {day}/{NUM_DAYS} days ({progress_pct:.1f}%)")
        print(f"   Total files so far: {total_so_far:,}")
        print(f"   Coverage: {total_so_far / expected_so_far * 100:.1f}%\n")

    except KeyboardInterrupt:
        print(f"\n\n⏸️  Download interrupted by user")
        print(f"   All progress saved in Google Drive/Lightning")
        print(f"   You can resume by running this script again.")
        break

    except Exception as e:
        print(f"❌ Error: {e}")
        failed_days.append(f"July {day}")
        print(f"   Continuing to next day...\n")
        continue

# ----------------------------------------------------------------------------
# FINAL SUMMARY
# ----------------------------------------------------------------------------

elapsed_time = time.time() - start_time
total_files = total_files_downloaded + total_files_existing

print("\n" + "=" * 70)
print("DOWNLOAD COMPLETE - JULY 2024")
print("=" * 70)

print(f"\n📊 Files:")
print(f"   New downloads: {total_files_downloaded:,}")
print(f"   Already existing: {total_files_existing:,}")
print(f"   Total: {total_files:,}")

expected_files = NUM_DAYS * 1440
coverage = (total_files / expected_files) * 100

print(f"\n📈 Coverage:")
print(f"   Expected: ~{expected_files:,} files")
print(f"   Actual: {total_files:,} files")
print(f"   Coverage: {coverage:.1f}%")

print(f"\n⏱️  Time:")
print(f"   Total elapsed: {elapsed_time/3600:.2f} hours")
if total_files_downloaded > 0:
    print(f"   Average speed: {elapsed_time/total_files_downloaded:.2f} sec/file")

# Storage info
if total_files > 0:
    sample_files = list(out_dir.glob("*.nc"))[:10]
    if sample_files:
        avg_file_size_mb = sum(f.stat().st_size for f in sample_files) / len(sample_files) / (1024**2)
        total_size_gb = (avg_file_size_mb * total_files) / 1024

        print(f"\n💾 Storage:")
        print(f"   Average file size: {avg_file_size_mb:.2f} MB")
        print(f"   Total size: {total_size_gb:.2f} GB")

print(f"\n📁 Location:")
print(f"   Google Drive → Lightning → glm_raw → 2024 → 07")
print(f"   Direct path: {out_dir}")

if failed_days:
    print(f"\n⚠️  Failed/Missing days ({len(failed_days)}):")
    for day in failed_days:
        print(f"   - {day}")
else:
    print(f"\n🎉 All 31 days downloaded successfully!")

print("\n" + "=" * 70)
print("NEXT STEPS")
print("=" * 70)
print("✅ Raw data saved to Google Drive/Lightning/glm_raw")
print("➡️  Step 1: Run hourly aggregation script (next)")
print("➡️  Step 2: Delete raw files (saves ~45 GB)")
print("➡️  Step 3: Load hourly files to Jupyter (~1.5 GB only)")
print("=" * 70 + "\n")

GLM DOWNLOAD TO GOOGLE DRIVE/Lightning - JULY 2024

🔐 Authenticating with NASA Earthdata...
✅ Authentication successful!

CONFIGURATION
📅 Target: July 2024 (31 days)
🗺️  Region: USA Continental (-130, 20, -60, 55)
📁 Output: G:/My Drive/Lightning/glm_raw/2024/07
💾 Expected size: ~40-50 GB
⏱️  Estimated time: 4-6 hours
📊 Available space in Drive: ~94 GB

🚀 Starting download to Google Drive/Lightning folder


📁 Files will be saved to:
   G:/My Drive/Lightning/glm_raw/2024/07

📅 Day 01/31: July 1, 2024
----------------------------------------------------------------------
🔍 Searching...
✅ Found: 2000 files
⬇️  Downloading to Google Drive/Lightning...


QUEUEING TASKS | :   0%|          | 0/2000 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2000 [00:00<?, ?it/s]



⏸️  Download interrupted by user
   All progress saved in Google Drive/Lightning
   You can resume by running this script again.

DOWNLOAD COMPLETE - JULY 2024

📊 Files:
   New downloads: 0
   Already existing: 0
   Total: 0

📈 Coverage:
   Expected: ~44,640 files
   Actual: 0 files
   Coverage: 0.0%

⏱️  Time:
   Total elapsed: 0.07 hours

📁 Location:
   Google Drive → Lightning → glm_raw → 2024 → 07
   Direct path: G:/My Drive/Lightning/glm_raw/2024/07

🎉 All 31 days downloaded successfully!

NEXT STEPS
✅ Raw data saved to Google Drive/Lightning/glm_raw
➡️  Step 1: Run hourly aggregation script (next)
➡️  Step 2: Delete raw files (saves ~45 GB)
➡️  Step 3: Load hourly files to Jupyter (~1.5 GB only)



In [1]:
# ============================================================================
# GLM-L3 HOURLY AGGREGATION - ERA5 GRID (0.25°)
# VERSIÓN PARALELA OPTIMIZADA PARA LEAP
# ============================================================================

import xarray as xr
import numpy as np
from pathlib import Path
from datetime import datetime
import time
import warnings
import sys
import os
from multiprocessing import Pool
import gc

warnings.filterwarnings("ignore")

print("=" * 70)
print("GLM-L3 → ERA5 GRID - PARALLEL ON LEAP")
print("=" * 70 + "\n")

# ============================================================================
# CONFIGURATION - PATHS CORREGIDOS PARA LEAP
# ============================================================================

# Path correcto en LEAP (con "G:" como carpeta literal)
RAW_DIR = Path("/home/jovyan/Lightning_Prediction/G:/My Drive/Lightning/glm_raw/2024/07")
HOURLY_DIR = Path("/home/jovyan/Lightning_Prediction/glm_hourly/2024/07")
HOURLY_DIR.mkdir(parents=True, exist_ok=True)

# USA Continental bounds
LON_MIN, LON_MAX = -130, -60
LAT_MIN, LAT_MAX = 20, 55

# ERA5 standard resolution
GRID_RES = 0.25

# Create ERA5 grid
lons_era5 = np.arange(LON_MIN, LON_MAX + GRID_RES, GRID_RES)
lats_era5 = np.arange(LAT_MIN, LAT_MAX + GRID_RES, GRID_RES)

# MÁS WORKERS en LEAP (tienes más recursos)
N_WORKERS = 6  # Puedes aumentar a 6-8 si quieres

print("CONFIGURATION")
print("-" * 70)
print(f"📁 Input:  {RAW_DIR}")
print(f"📁 Output: {HOURLY_DIR}")
print(f"🗺️  Region: USA Continental")
print(f"   Lon: {LON_MIN} to {LON_MAX}")
print(f"   Lat: {LAT_MIN} to {LAT_MAX}")
print(f"📊 Grid: ERA5 {GRID_RES}° resolution")
print(f"   {len(lons_era5)} lons × {len(lats_era5)} lats = {len(lons_era5)*len(lats_era5):,} cells")
print(f"📅 Period: July 2024 (744 hours)")
print(f"🚀 Workers: {N_WORKERS} parallel processes on LEAP\n")

# Verificar que los datos existen
if not RAW_DIR.exists():
    print(f"❌ ERROR: {RAW_DIR} no existe!")
    sys.exit(1)

num_files = len(list(RAW_DIR.glob("*.nc")))
print(f"✅ Verificado: {num_files:,} archivos NetCDF encontrados\n")

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def get_hour_string(year, month, day, hour):
    return f"{year}{month:02d}{day:02d}_{hour:02d}00"

def get_files_for_hour(year, month, day, hour):
    """Get all GLM files for a specific hour"""
    day_of_year = datetime(year, month, day).timetuple().tm_yday
    files = []
    for minute in range(60):
        pattern = f"*_s{year}{day_of_year:03d}{hour:02d}{minute:02d}*.nc"
        files.extend(RAW_DIR.glob(pattern))
    return sorted(files)

def glm_xy_to_lonlat(x, y, proj_info):
    """Convert GLM geostationary projection (x,y) to geographic (lon,lat)"""
    sat_height = proj_info.get('perspective_point_height', 35786023.0)
    sat_lon = proj_info.get('longitude_of_projection_origin', -75.0)
    semi_major = proj_info.get('semi_major_axis', 6378137.0)
    semi_minor = proj_info.get('semi_minor_axis', 6356752.31414)
    
    H = sat_height + semi_major
    
    cos_x = np.cos(x)
    cos_y = np.cos(y)
    sin_x = np.sin(x)
    sin_y = np.sin(y)
    
    a = cos_x**2 + (semi_major/semi_minor)**2 * sin_x**2
    b = -2 * H * cos_y * cos_x
    c = H**2 - semi_major**2
    
    r_s = (-b - np.sqrt(b**2 - 4*a*c)) / (2*a)
    
    s_x = r_s * cos_y * cos_x
    s_y = -r_s * sin_y
    s_z = r_s * cos_y * sin_x
    
    lon = sat_lon - np.degrees(np.arctan(s_y / (H - s_x)))
    lat = np.degrees(np.arctan((semi_major/semi_minor)**2 * s_z / np.sqrt((H - s_x)**2 + s_y**2)))
    
    return lon, lat

def aggregate_hour_to_era5_grid(files):
    """Aggregate GLM data to ERA5 grid - MEMORY OPTIMIZED"""
    if len(files) == 0:
        return None
    
    # Suppress stderr
    old_stderr = sys.stderr
    sys.stderr = open(os.devnull, 'w')
    
    try:
        # Initialize output arrays
        flash_density = np.zeros((len(lats_era5), len(lons_era5)), dtype=np.float32)
        total_energy = np.zeros((len(lats_era5), len(lons_era5)), dtype=np.float64)
        
        files_processed = 0
        coordinate_mapping_done = False
        
        # Process files one at a time
        for file in files:
            try:
                with xr.open_dataset(file) as ds:
                    # Only compute coordinate mapping once
                    if not coordinate_mapping_done:
                        x_coords = ds['x'].values
                        y_coords = ds['y'].values
                        proj = ds['goes_imager_projection']
                        
                        # Create meshgrid
                        x_2d, y_2d = np.meshgrid(x_coords, y_coords)
                        
                        # Convert to lon/lat
                        lon_2d, lat_2d = glm_xy_to_lonlat(x_2d, y_2d, proj.attrs)
                        
                        # Find cells within USA bounds
                        usa_mask = (lon_2d >= LON_MIN) & (lon_2d <= LON_MAX) & \
                                   (lat_2d >= LAT_MIN) & (lat_2d <= LAT_MAX)
                        
                        # Get indices and coordinates of USA cells
                        y_idx, x_idx = np.where(usa_mask)
                        lon_vals = lon_2d[usa_mask]
                        lat_vals = lat_2d[usa_mask]
                        
                        # Pre-compute ERA5 grid indices for each GLM cell
                        lon_grid_idx = ((lon_vals - LON_MIN) / GRID_RES).astype(int)
                        lat_grid_idx = ((lat_vals - LAT_MIN) / GRID_RES).astype(int)
                        
                        # Clip to bounds
                        lon_grid_idx = np.clip(lon_grid_idx, 0, len(lons_era5) - 1)
                        lat_grid_idx = np.clip(lat_grid_idx, 0, len(lats_era5) - 1)
                        
                        coordinate_mapping_done = True
                        
                        # Free memory
                        del x_2d, y_2d, lon_2d, lat_2d, usa_mask
                        gc.collect()
                    
                    # Extract data for USA region only
                    flash_data = ds['Flash_extent_density'].values[y_idx, x_idx]
                    energy_data = ds['Total_Optical_energy'].values[y_idx, x_idx]
                    
                    # Only process non-NaN cells
                    valid = ~np.isnan(flash_data)
                    
                    if np.any(valid):
                        # Accumulate to ERA5 grid
                        np.add.at(flash_density, (lat_grid_idx[valid], lon_grid_idx[valid]), flash_data[valid])
                        np.add.at(total_energy, (lat_grid_idx[valid], lon_grid_idx[valid]), energy_data[valid])
                    
                    files_processed += 1
                    
                    # Free memory after each file
                    del flash_data, energy_data, valid
                    
            except Exception:
                continue
        
        # Clean up
        gc.collect()
        
        total_flashes = int(np.sum(flash_density))
        
        return {
            'flash_density': flash_density,
            'total_energy': total_energy,
            'total_flashes': total_flashes,
            'files_processed': files_processed
        }
    
    finally:
        sys.stderr.close()
        sys.stderr = old_stderr
        gc.collect()

def save_hourly_netcdf(data, year, month, day, hour, output_dir):
    """Save hourly data in ERA5-compatible format"""
    timestamp = datetime(year, month, day, hour)
    
    ds = xr.Dataset(
        data_vars={
            "lightning_density": (
                ["latitude", "longitude"],
                data["flash_density"],
                {
                    "long_name": "Lightning flash density",
                    "units": "flashes",
                    "description": "Hourly accumulated flash extent density"
                }
            ),
            "lightning_energy": (
                ["latitude", "longitude"],
                data["total_energy"],
                {
                    "long_name": "Lightning optical energy",
                    "units": "J",
                    "description": "Hourly accumulated total optical energy"
                }
            ),
        },
        coords={
            "longitude": (["longitude"], lons_era5, {"units": "degrees_east", "long_name": "Longitude"}),
            "latitude": (["latitude"], lats_era5, {"units": "degrees_north", "long_name": "Latitude"}),
            "time": timestamp,
        },
        attrs={
            "title": "GLM Lightning Data on ERA5 Grid",
            "source": "GOES-16 GLM Level 3",
            "grid_resolution": "0.25 degrees (ERA5 standard)",
            "spatial_coverage": f"USA Continental ({LON_MIN}, {LAT_MIN}) to ({LON_MAX}, {LAT_MAX})",
            "temporal_resolution": "1 hour",
            "created": datetime.now().isoformat(),
            "total_flashes": data["total_flashes"],
            "files_processed": data["files_processed"],
            "ML_ready": "Compatible with ERA5 grid for machine learning"
        },
    )
    
    encoding = {
        "lightning_density": {"zlib": True, "complevel": 5},
        "lightning_energy": {"zlib": True, "complevel": 5},
    }
    
    filename = f"GLM_ERA5grid_hourly_{get_hour_string(year, month, day, hour)}.nc"
    filepath = output_dir / filename
    ds.to_netcdf(filepath, encoding=encoding)
    ds.close()
    
    # Free memory
    del ds
    gc.collect()

# ============================================================================
# PARALLEL PROCESSING
# ============================================================================

def process_single_hour(hour_info):
    """Process one hour - memory safe"""
    year, month, day, hour = hour_info
    
    try:
        # Check if already exists
        filename = f"GLM_ERA5grid_hourly_{get_hour_string(year, month, day, hour)}.nc"
        out_path = HOURLY_DIR / filename
        
        if out_path.exists():
            return (day, hour, "exists", 0, 0, 0)
        
        # Get files
        files = get_files_for_hour(year, month, day, hour)
        
        if len(files) == 0:
            return (day, hour, "no_files", 0, 0, 0)
        
        # Process
        start = time.time()
        data = aggregate_hour_to_era5_grid(files)
        
        if data is None:
            return (day, hour, "failed", 0, 0, 0)
        
        # Save
        save_hourly_netcdf(data, year, month, day, hour, HOURLY_DIR)
        
        elapsed = time.time() - start
        
        total_flashes = data['total_flashes']
        n_files = data['files_processed']
        
        # Clean up
        del data, files
        gc.collect()
        
        return (day, hour, "success", total_flashes, n_files, elapsed)
        
    except Exception as e:
        return (day, hour, "error", 0, 0, 0)

# ============================================================================
# MAIN - PROCESS DAY BY DAY
# ============================================================================

if __name__ == '__main__':
    print("=" * 70)
    print("PROCESSING (PARALLEL BY DAY)")
    print("=" * 70 + "\n")
    
    YEAR = 2024
    MONTH = 7
    NUM_DAYS = 31
    
    start_time = time.time()
    total_flashes_all = 0
    total_files_all = 0
    hours_processed = 0
    
    # PROCESS ONE DAY AT A TIME
    for day in range(1, NUM_DAYS + 1):
        print(f"📅 Day {day:02d}/31: July {day}, {YEAR}")
        print("-" * 70)
        
        # Prepare hours for this day
        day_hours = [(YEAR, MONTH, day, hour) for hour in range(24)]
        
        # Process this day in parallel
        with Pool(N_WORKERS) as pool:
            results = pool.map(process_single_hour, day_hours)
        
        # Display results for this day
        for day_num, hour, status, flashes, n_files, elapsed in results:
            hours_processed += 1
            progress = f"{hours_processed}/744 ({100*hours_processed/744:.1f}%)"
            
            if status == "exists":
                print(f"   Hour {hour:02d}:00 - ✓ Exists [{progress}]")
            elif status == "no_files":
                print(f"   Hour {hour:02d}:00 - ⚠ No files [{progress}]")
            elif status == "failed":
                print(f"   Hour {hour:02d}:00 - ✗ Failed [{progress}]")
            elif status == "error":
                print(f"   Hour {hour:02d}:00 - ✗ Error [{progress}]")
            else:  # success
                total_flashes_all += flashes
                total_files_all += n_files
                print(f"   Hour {hour:02d}:00 - ✓ {flashes:,} flashes, {n_files} files ({elapsed:.1f}s) [{progress}]")
        
        print()  # Blank line after each day
        
        # Force garbage collection after each day
        gc.collect()
    
    # ========================================================================
    # FINAL SUMMARY
    # ========================================================================
    
    elapsed_time = time.time() - start_time
    
    print("\n" + "=" * 70)
    print("AGGREGATION COMPLETE")
    print("=" * 70)
    
    print(f"\n📊 Processing:")
    print(f"   Hours processed: {hours_processed}/744")
    print(f"   Total flashes: {total_flashes_all:,}")
    print(f"   Raw files: {total_files_all:,}")
    
    print(f"\n⏱️  Time:")
    print(f"   Total: {elapsed_time/3600:.2f} hours")
    if hours_processed > 0:
        print(f"   Average: {elapsed_time/hours_processed:.1f} sec/hour")
    
    # Storage stats
    hourly_files = list(HOURLY_DIR.glob("GLM_ERA5grid_*.nc"))
    if hourly_files:
        total_size_mb = sum(f.stat().st_size for f in hourly_files) / (1024**2)
        
        print(f"\n💾 Storage:")
        print(f"   Files: {len(hourly_files)}")
        print(f"   Total: {total_size_mb/1024:.2f} GB")
        print(f"   Average: {total_size_mb/len(hourly_files):.2f} MB/file")
    
    print(f"\n🎯 Output:")
    print(f"   Location: {HOURLY_DIR}")
    print(f"   Format: NetCDF on ERA5 0.25° grid")
    print(f"   Grid: {len(lons_era5)} × {len(lats_era5)} cells")
    print(f"   ✅ Ready for ML with ERA5 data")
    
    print("\n" + "=" * 70)

GLM-L3 → ERA5 GRID - PARALLEL ON LEAP

CONFIGURATION
----------------------------------------------------------------------
📁 Input:  /home/jovyan/Lightning_Prediction/G:/My Drive/Lightning/glm_raw/2024/07
📁 Output: /home/jovyan/Lightning_Prediction/glm_hourly/2024/07
🗺️  Region: USA Continental
   Lon: -130 to -60
   Lat: 20 to 55
📊 Grid: ERA5 0.25° resolution
   281 lons × 141 lats = 39,621 cells
📅 Period: July 2024 (744 hours)
🚀 Workers: 6 parallel processes on LEAP

❌ ERROR: /home/jovyan/Lightning_Prediction/G:/My Drive/Lightning/glm_raw/2024/07 no existe!


SystemExit: 1